In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../data/naver_shopping_tokenized_review.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Unnamed: 0   200000 non-null  int64 
 1   Score        200000 non-null  int64 
 2   Review       200000 non-null  object
 3   y            200000 non-null  int64 
 4   tokenized_1  199572 non-null  object
 5   tokenized_2  199985 non-null  object
dtypes: int64(3), object(3)
memory usage: 9.2+ MB


In [4]:
df = df[df['tokenized_1'].notnull() & df['tokenized_2'].notnull()]

### DTM prediction column 추가(tokenizer1)
* 예측치 of (DTM + 명사, 형용사, 동사만 추출 & 2글자 이상만 토큰화)

In [5]:
from sklearn.model_selection import train_test_split
y = df['y']
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized_1'], y, test_size = 0.2, random_state = 0)

In [7]:
def tokenizer(text):
    return text.split()

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df = 3, ngram_range=(1,2), tokenizer = tokenizer)
X_train_tf = vect.fit_transform(X_train_texts)
X_total = vect.transform(df['tokenized_1'])

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
import pickle
with open('DTM_tokenizer1_gridsearch_done.pickle','rb') as f:
    lr_DTM_tk1 = pickle.load(f)

In [10]:
y_pred = lr_DTM_tk1.predict(X_total)

In [12]:
# del df[df.columns[0]]
df.head()

,Score,Review,y,tokenized_1,tokenized_2
0,5,배공빠르고 굿,1,배공 빠르고,배공 빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,택배 엉망 놔두고가고,택배 엉망 이네 용 집 밑 층 없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 박음질이 조금 ...,1,아주 좋아요 바지 정말 좋아서 구매 가격 대박 입니다 박음질 어설프다하긴 편하고 가...,아주 좋아요 바지 정말 좋아서 2 구매 가격 대박 입니다 . 박음질 어설프다하긴 편...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,선물 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 전화했더니 ...,선물 용 빨리 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 ....
4,5,민트색상상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,민트 색상 예뻐요 손잡이 도로 사용 되네요,민트 색상 예뻐요 . 손잡이 는 는 용 도로 사용 되네요 ㅎㅎ


In [13]:
df['DTM_tk1_prediction'] = y_pred

### TFIDF prediction column 추가(tokenizer1)
* 예측치 of (TFIDF + 명사, 형용사, 동사만 추출 & 2글자 이상만 토큰화)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = tokenizer)
tfidf_vectorizer.fit(X_train_texts)
tfidf_matrix_X_total = tfidf_vectorizer.transform(df['tokenized_1'])

In [16]:
with open('TFIDF_tokenizer1_gridsearch_done.pickle','rb') as f:
    lr_TFIDF_tk1 = pickle.load(f)

In [17]:
y_pred2 = lr_TFIDF_tk1.predict(tfidf_matrix_X_total)

In [18]:
df['TFIDF_tk1_prediction'] = y_pred2

In [19]:
df.head()

,Score,Review,y,tokenized_1,tokenized_2,DTM_tk1_prediction,TFIDF_tk1_prediction
0,5,배공빠르고 굿,1,배공 빠르고,배공 빠르고 굿,1,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,택배 엉망 놔두고가고,택배 엉망 이네 용 집 밑 층 없이 놔두고가고,0,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 박음질이 조금 ...,1,아주 좋아요 바지 정말 좋아서 구매 가격 대박 입니다 박음질 어설프다하긴 편하고 가...,아주 좋아요 바지 정말 좋아서 2 구매 가격 대박 입니다 . 박음질 어설프다하긴 편...,1,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,선물 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 전화했더니 ...,선물 용 빨리 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 ....,0,0
4,5,민트색상상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,민트 색상 예뻐요 손잡이 도로 사용 되네요,민트 색상 예뻐요 . 손잡이 는 는 용 도로 사용 되네요 ㅎㅎ,1,1


### DTM prediction column 추가(tokenizer2)
* 예측치 of (DTM + 형태소 모두 활용)

In [20]:
X_train_texts, X_test_texts, y_train, y_test = train_test_split(df['tokenized_2'], y, test_size = 0.2, random_state = 0)

In [21]:
vect2 = CountVectorizer(min_df = 3, ngram_range=(1,2), tokenizer = tokenizer)
X_train_tf = vect2.fit_transform(X_train_texts)
X_total_tf = vect2.transform(df['tokenized_2'])

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [27]:
with open('DTM_tokenizer2_gridsearch_done.pickle','rb') as f:
    lr_DTM_tk2 = pickle.load(f)

In [28]:
y_pred3 = lr_DTM_tk2.predict(X_total_tf)

In [29]:
df['DTM_tk2_prediction'] = y_pred3

In [30]:
df.head()

,Score,Review,y,tokenized_1,tokenized_2,DTM_tk1_prediction,TFIDF_tk1_prediction,DTM_tk2_prediction
0,5,배공빠르고 굿,1,배공 빠르고,배공 빠르고 굿,1,1,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0,택배 엉망 놔두고가고,택배 엉망 이네 용 집 밑 층 없이 놔두고가고,0,0,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 박음질이 조금 ...,1,아주 좋아요 바지 정말 좋아서 구매 가격 대박 입니다 박음질 어설프다하긴 편하고 가...,아주 좋아요 바지 정말 좋아서 2 구매 가격 대박 입니다 . 박음질 어설프다하긴 편...,1,1,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0,선물 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 전화했더니 ...,선물 용 빨리 받아서 전달 했어야 하는 상품 이었는데 머그컵 와서 당황 했습니다 ....,0,0,0
4,5,민트색상상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1,민트 색상 예뻐요 손잡이 도로 사용 되네요,민트 색상 예뻐요 . 손잡이 는 는 용 도로 사용 되네요 ㅎㅎ,1,1,1


### TFIDF prediction column 추가(tokenizer2)
* 예측치 of (TFIDF + 형태소 모두 활용)

In [26]:
tfidf_vectorizer2 = TfidfVectorizer(ngram_range=(1,2), min_df = 3, max_df = 0.9, tokenizer = tokenizer)
tfidf_vectorizer2.fit(X_train_texts)
tfidf_matrix_X_total = tfidf_vectorizer2.transform(df['tokenized_2'])

c:\Users\taeri\miniconda3\envs\ds_study\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [33]:
with open('TFIDF_tokenizer2_gridsearch_done.pickle','rb') as f:
    lr_TFIDF_tk2 = pickle.load(f)

In [34]:
y_pred4 = lr_TFIDF_tk2.predict(tfidf_matrix_X_total)

In [35]:
df['TFIDF_tk2_prediction'] = y_pred4

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199572 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Score                 199572 non-null  int64 
 1   Review                199572 non-null  object
 2   y                     199572 non-null  int64 
 3   tokenized_1           199572 non-null  object
 4   tokenized_2           199572 non-null  object
 5   DTM_tk1_prediction    199572 non-null  int64 
 6   TFIDF_tk1_prediction  199572 non-null  int64 
 7   DTM_tk2_prediction    199572 non-null  int64 
 8   TFIDF_tk2_prediction  199572 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 15.2+ MB


In [37]:
df.to_csv("../../data/naver_shopping_tokenized_review_and_prediction_by_models.csv", sep = ",", encoding ="utf-8")

y  DTM_tk1_prediction  TFIDF_tk1_prediction
0  0                   0                       92987
   1                   1                        5471
   0                   1                         841
   1                   0                         506
1  1                   1                       92175
   0                   0                        5486
   1                   0                        1148
   0                   1                         958
dtype: int64